In [6]:
from torch.optim import lr_scheduler
import sys
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F
from torch import Tensor
from typing import List, Callable,Optional
import math
import copy
from functools import partial
from collections import OrderedDict

# 加载规范化数据集
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

train_dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/TypeMasks/train', transform=train_transform)
val_dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/TypeMasks/test', transform=val_transform)
train_num =  len(train_dataset)
val_num = len(val_dataset)

batch_size =4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print("Loading completed")





Loading completed


In [7]:
# model
def _make_divisible(ch, divisor=8, min_ch=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_ch is None:
        min_ch = divisor
    new_ch = max(min_ch, int(ch + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_ch < 0.9 * ch:
        new_ch += divisor
    return new_ch


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    "Deep Networks with Stochastic Depth", https://arxiv.org/pdf/1603.09382.pdf

    This function is taken from the rwightman.
    It can be seen here:
    https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/layers/drop.py#L140
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    "Deep Networks with Stochastic Depth", https://arxiv.org/pdf/1603.09382.pdf
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class ConvBNActivation(nn.Sequential):
    def __init__(self,
                 in_planes: int,
                 out_planes: int,
                 kernel_size: int = 3,
                 stride: int = 1,
                 groups: int = 1,
                 norm_layer: Optional[Callable[..., nn.Module]] = None,
                 activation_layer: Optional[Callable[..., nn.Module]] = None):
        padding = (kernel_size - 1) // 2
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if activation_layer is None:
            activation_layer = nn.SiLU  # alias Swish  (torch>=1.7)

        super(ConvBNActivation, self).__init__(nn.Conv2d(in_channels=in_planes,
                                                         out_channels=out_planes,
                                                         kernel_size=kernel_size,
                                                         stride=stride,
                                                         padding=padding,
                                                         groups=groups,
                                                         bias=False),
                                               norm_layer(out_planes),
                                               activation_layer())


class SqueezeExcitation(nn.Module):
    def __init__(self,
                 input_c: int,   # block input channel
                 expand_c: int,  # block expand channel
                 squeeze_factor: int = 4):
        super(SqueezeExcitation, self).__init__()
        squeeze_c = input_c // squeeze_factor
        self.fc1 = nn.Conv2d(expand_c, squeeze_c, 1)
        self.ac1 = nn.SiLU()  # alias Swish
        self.fc2 = nn.Conv2d(squeeze_c, expand_c, 1)
        self.ac2 = nn.Sigmoid()

    def forward(self, x: Tensor) -> Tensor:
        scale = F.adaptive_avg_pool2d(x, output_size=(1, 1))
        scale = self.fc1(scale)
        scale = self.ac1(scale)
        scale = self.fc2(scale)
        scale = self.ac2(scale)
        return scale * x


class InvertedResidualConfig:
    # kernel_size, in_channel, out_channel, exp_ratio, strides, use_SE, drop_connect_rate
    def __init__(self,
                 kernel: int,          # 3 or 5
                 input_c: int,
                 out_c: int,
                 expanded_ratio: int,  # 1 or 6
                 stride: int,          # 1 or 2
                 use_se: bool,         # True
                 drop_rate: float,
                 index: str,           # 1a, 2a, 2b, ...
                 width_coefficient: float):
        self.input_c = self.adjust_channels(input_c, width_coefficient)
        self.kernel = kernel
        self.expanded_c = self.input_c * expanded_ratio
        self.out_c = self.adjust_channels(out_c, width_coefficient)
        self.use_se = use_se
        self.stride = stride
        self.drop_rate = drop_rate
        self.index = index

    @staticmethod
    def adjust_channels(channels: int, width_coefficient: float):
        return _make_divisible(channels * width_coefficient, 8)


class InvertedResidual(nn.Module):
    def __init__(self,
                 cnf: InvertedResidualConfig,
                 norm_layer: Callable[..., nn.Module]):
        super(InvertedResidual, self).__init__()

        if cnf.stride not in [1, 2]:
            raise ValueError("illegal stride value.")

        self.use_res_connect = (cnf.stride == 1 and cnf.input_c == cnf.out_c)

        layers = OrderedDict()
        activation_layer = nn.SiLU  # alias Swish

        # expand
        if cnf.expanded_c != cnf.input_c:
            layers.update({"expand_conv": ConvBNActivation(cnf.input_c,
                                                           cnf.expanded_c,
                                                           kernel_size=1,
                                                           norm_layer=norm_layer,
                                                           activation_layer=activation_layer)})

        # depthwise
        layers.update({"dwconv": ConvBNActivation(cnf.expanded_c,
                                                  cnf.expanded_c,
                                                  kernel_size=cnf.kernel,
                                                  stride=cnf.stride,
                                                  groups=cnf.expanded_c,
                                                  norm_layer=norm_layer,
                                                  activation_layer=activation_layer)})

        if cnf.use_se:
            layers.update({"se": SqueezeExcitation(cnf.input_c,
                                                   cnf.expanded_c)})

        # project
        layers.update({"project_conv": ConvBNActivation(cnf.expanded_c,
                                                        cnf.out_c,
                                                        kernel_size=1,
                                                        norm_layer=norm_layer,
                                                        activation_layer=nn.Identity)})

        self.block = nn.Sequential(layers)
        self.out_channels = cnf.out_c
        self.is_strided = cnf.stride > 1

        # 只有在使用shortcut连接时才使用dropout层
        if self.use_res_connect and cnf.drop_rate > 0:
            self.dropout = DropPath(cnf.drop_rate)
        else:
            self.dropout = nn.Identity()

    def forward(self, x: Tensor) -> Tensor:
        result = self.block(x)
        result = self.dropout(result)
        if self.use_res_connect:
            result += x

        return result


class EfficientNet(nn.Module):
    def __init__(self,
                 width_coefficient: float,
                 depth_coefficient: float,
                 num_classes: int = 1000,
                 dropout_rate: float = 0.2,
                 drop_connect_rate: float = 0.2,
                 block: Optional[Callable[..., nn.Module]] = None,
                 norm_layer: Optional[Callable[..., nn.Module]] = None
                 ):
        super(EfficientNet, self).__init__()

        # kernel_size, in_channel, out_channel, exp_ratio, strides, use_SE, drop_connect_rate, repeats
        default_cnf = [[3, 32, 16, 1, 1, True, drop_connect_rate, 1],
                       [3, 16, 24, 6, 2, True, drop_connect_rate, 2],
                       [5, 24, 40, 6, 2, True, drop_connect_rate, 2],
                       [3, 40, 80, 6, 2, True, drop_connect_rate, 3],
                       [5, 80, 112, 6, 1, True, drop_connect_rate, 3],
                       [5, 112, 192, 6, 2, True, drop_connect_rate, 4],
                       [3, 192, 320, 6, 1, True, drop_connect_rate, 1]]

        def round_repeats(repeats):
            """Round number of repeats based on depth multiplier."""
            return int(math.ceil(depth_coefficient * repeats))

        if block is None:
            block = InvertedResidual

        if norm_layer is None:
            norm_layer = partial(nn.BatchNorm2d, eps=1e-3, momentum=0.1)

        adjust_channels = partial(InvertedResidualConfig.adjust_channels,
                                  width_coefficient=width_coefficient)

        # build inverted_residual_setting
        bneck_conf = partial(InvertedResidualConfig,
                             width_coefficient=width_coefficient)

        b = 0
        num_blocks = float(sum(round_repeats(i[-1]) for i in default_cnf))
        inverted_residual_setting = []
        for stage, args in enumerate(default_cnf):
            cnf = copy.copy(args)
            for i in range(round_repeats(cnf.pop(-1))):
                if i > 0:
                    # strides equal 1 except first cnf
                    cnf[-3] = 1  # strides
                    cnf[1] = cnf[2]  # input_channel equal output_channel

                cnf[-1] = args[-2] * b / num_blocks  # update dropout ratio
                index = str(stage + 1) + chr(i + 97)  # 1a, 2a, 2b, ...
                inverted_residual_setting.append(bneck_conf(*cnf, index))
                b += 1

        # create layers
        layers = OrderedDict()

        # first conv
        layers.update({"stem_conv": ConvBNActivation(in_planes=3,
                                                     out_planes=adjust_channels(32),
                                                     kernel_size=3,
                                                     stride=2,
                                                     norm_layer=norm_layer)})

        # building inverted residual blocks
        for cnf in inverted_residual_setting:
            layers.update({cnf.index: block(cnf, norm_layer)})

        # build top
        last_conv_input_c = inverted_residual_setting[-1].out_c
        last_conv_output_c = adjust_channels(1280)
        layers.update({"top": ConvBNActivation(in_planes=last_conv_input_c,
                                               out_planes=last_conv_output_c,
                                               kernel_size=1,
                                               norm_layer=norm_layer)})

        self.features = nn.Sequential(layers)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        classifier = []
        if dropout_rate > 0:
            classifier.append(nn.Dropout(p=dropout_rate, inplace=True))
        classifier.append(nn.Linear(last_conv_output_c, num_classes))
        self.classifier = nn.Sequential(*classifier)

        # initial weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def _forward_impl(self, x: Tensor) -> Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def efficientnet_b0(num_classes=1000):
    # input image size 224x224
    return EfficientNet(width_coefficient=1.0,
                        depth_coefficient=1.0,
                        dropout_rate=0.2,
                        num_classes=num_classes)


def efficientnet_b1(num_classes=1000):
    # input image size 240x240
    return EfficientNet(width_coefficient=1.0,
                        depth_coefficient=1.1,
                        dropout_rate=0.2,
                        num_classes=num_classes)


def efficientnet_b2(num_classes=1000):
    # input image size 260x260
    return EfficientNet(width_coefficient=1.1,
                        depth_coefficient=1.2,
                        dropout_rate=0.3,
                        num_classes=num_classes)


def efficientnet_b3(num_classes=1000):
    # input image size 300x300
    return EfficientNet(width_coefficient=1.2,
                        depth_coefficient=1.4,
                        dropout_rate=0.3,
                        num_classes=num_classes)


def efficientnet_b4(num_classes=1000):
    # input image size 380x380
    return EfficientNet(width_coefficient=1.4,
                        depth_coefficient=1.8,
                        dropout_rate=0.4,
                        num_classes=num_classes)


def efficientnet_b5(num_classes=1000):
    # input image size 456x456
    return EfficientNet(width_coefficient=1.6,
                        depth_coefficient=2.2,
                        dropout_rate=0.4,
                        num_classes=num_classes)


def efficientnet_b6(num_classes=1000):
    # input image size 528x528
    return EfficientNet(width_coefficient=1.8,
                        depth_coefficient=2.6,
                        dropout_rate=0.5,
                        num_classes=num_classes)


def efficientnet_b7(num_classes=1000):
    # input image size 600x600
    return EfficientNet(width_coefficient=2.0,
                        depth_coefficient=3.1,
                        dropout_rate=0.5,
                        num_classes=num_classes)

In [8]:
# train
def train(dataloader, model, loss_fn, optimizer):
    loss, current, n = 0.0, 0.0, 0
    for batch, (x, y) in enumerate(dataloader):
        image, y = x.to(device), y.to(device)
        output = model(image)
        cur_loss = loss_fn(output, y)
        _, pred = torch.max(output, axis=1)
        cur_acc = torch.sum(y==pred) / output.shape[0]

        # 反向传播
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        loss += cur_loss.item()
        current += cur_acc.item()
        n = n+1

    train_loss = loss / n
    train_acc = current / n
    print('train_loss' + str(train_loss))
    print('train_acc' + str(train_acc))
    return train_loss, train_acc

# 定义一个验证函数
def val(dataloader, model, loss_fn):
    # 将模型转化为验证模型
    model.eval()
    loss, current, n = 0.0, 0.0, 0
    with torch.no_grad():
        for batch, (x, y) in enumerate(dataloader):
            image, y = x.to(device), y.to(device)
            output = model(image)
            cur_loss = loss_fn(output, y)
            _, pred = torch.max(output, axis=1)
            cur_acc = torch.sum(y == pred) / output.shape[0]
            loss += cur_loss.item()
            current += cur_acc.item()
            n = n + 1

    val_loss = loss / n
    val_acc = current / n
    print('val_loss' + str(val_loss))
    print('val_acc' + str(val_acc))
    return val_loss, val_acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

# 实例化model
model =  efficientnet_b0(num_classes=5)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
#lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
#checkpoint = torch.load(r"C:\Users\2021\Desktop\lib8\save_model\vgg11-bbd30ac9.pth")
#model.load_state_dict(checkpoint)
loss_train = []
acc_train = []
loss_val = []
acc_val = []
epochs = 20
for t in range(epochs):
    # lr_scheduler.step()
    print(f"epoch{t+1}\n-----------")
    train_loss, train_acc = train(train_dataloader, model, loss_function, optimizer)
    val_loss, val_acc = val(val_dataloader, model, loss_function)

    loss_train.append(train_loss)
    acc_train.append(train_acc)
    loss_val.append(val_loss)
    acc_val.append(val_acc)

    # 保存最好的模型权重
    # if val_acc >min_acc:
    print(f"save model, 第{t+1}轮")
    torch.save(model.state_dict(), r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/proj/efficientnet/save_model_typemask/epoch_'+str(t)+'_best_model.pth')


print('Finished Training')

using cpu device.
epoch1
-----------
train_loss0.8575017464558283
train_acc0.6933333333333334
val_loss0.07726520399855291
val_acc0.9893617021276596
save model, 第1轮
epoch2
-----------
train_lossnan
train_acc0.9426666666666667
val_lossnan
val_acc0.3324468085106383
save model, 第2轮
epoch3
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3337765957446808
save model, 第3轮
epoch4
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3324468085106383
save model, 第4轮
epoch5
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3351063829787234
save model, 第5轮
epoch6
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3351063829787234
save model, 第6轮
epoch7
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3324468085106383
save model, 第7轮
epoch8
-----------
train_lossnan
train_acc0.3333333333333333
val_lossnan
val_acc0.3324468085106383
save model, 第8轮
epoch9
-----------
train_lossna

In [11]:
#test
print('start test')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
data_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load image
dataset = ImageFolder(r'/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/projectdataset/TypeMasks/test', transform=train_transform)
datasetloader = DataLoader(dataset, batch_size=32, shuffle=True)

# create model
# model = vgg(model_name="vgg11", num_classes=1000).to(device)
weights_path = r"/Users/kkkkshi/Desktop/ANLY_677/Project/Project_3/proj/efficientnet/save_model_typemask/epoch_19_best_model.pth"
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()

top1 =0
top5= 0
data_num = len(dataset)
acc_num =0;
wrong_num = 0;
for i in range(data_num):
    x, y = dataset[i][0], dataset[i][1]
    x = torch.unsqueeze(x, dim=0)

    with torch.no_grad():
        # predict class
        output = torch.squeeze(model(x.to(device))).cpu()
        predict = torch.softmax(output, dim=0)

        predict_cla = torch.argmax(predict).numpy()
        print("predict:",predict_cla,"actual:",y)
        if predict_cla == y:
            acc_num +=1
wrong_num = data_num-acc_num
print("正确数量：",acc_num)
print("错误数量：",wrong_num)


def evaluteTop1(model, loader):
    model.eval()

    correct = 0
    total = len(loader.dataset)

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
            correct += torch.eq(pred, y).sum().float().item()
    return correct / total


def evaluteTop5(model, loader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            maxk = max((1, 5))
            y_resize = y.view(-1, 1)
            _, pred = logits.topk(maxk, 1, True, True)
            correct += torch.eq(pred, y_resize).sum().float().item()
    return correct / total
print("top1 accuracy: ",evaluteTop1(model,datasetloader))
print("top5 accuracy: ",evaluteTop5(model,datasetloader))

start test
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
predict: 0 actual: 0
pr

predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 actual: 1
predict: 0 ac